# Setup

In [218]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

In [219]:
sys.path.append(os.path.abspath("../../.."))

In [220]:
from src.experiment.helpers.variables import dataset_root_dir
from src.experiment.helpers.utils import get_sorted_class_percentages_label_encoded

In [221]:
NAME = "news-sarcasm"
CLASSIFICATION_TYPE = "binary"

In [222]:
DATASET_DIR = dataset_root_dir / CLASSIFICATION_TYPE / NAME
print(DATASET_DIR)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\binary\news-sarcasm


In [223]:
balanced_name = NAME + "_balanced"
DATASET_DIR_BALANCED = dataset_root_dir / CLASSIFICATION_TYPE / balanced_name
print(DATASET_DIR_BALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\binary\news-sarcasm_balanced


In [224]:
imbalanced_name = NAME + "_imbalanced"
DATASET_DIR_IMBALANCED = dataset_root_dir / CLASSIFICATION_TYPE / imbalanced_name
print(DATASET_DIR_IMBALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\binary\news-sarcasm_imbalanced


# Dataset loading

In [225]:
X = pd.read_csv(DATASET_DIR / 'X.csv')
y = pd.read_csv(DATASET_DIR / 'y.csv')

In [226]:
X

,000,10,100,11,12,13,14,15,16,17,...,ever,every,everyone,everything,evidence,ex,exactly,exchange,excited,exclusive
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28614,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [227]:
y

,is_sarcastic
0,1
1,0
2,0
3,1
4,1
...,...
28614,1
28615,1
28616,0
28617,1


# Cut classes to match balanced dataset

In [100]:
y_balanced = pd.read_csv(DATASET_DIR_BALANCED / 'y.csv')
y_balanced

,class
0,0
1,0
2,0
3,0
4,0
...,...
475,3
476,3
477,3
478,3


In [101]:
# Get the set of classes present in y_balanced
classes_in_balanced = set(y_balanced['class'].unique())
print(f"Classes in balanced dataset: {classes_in_balanced}")

# Create mask for samples in y that are present in y_balanced
mask = y['class'].isin(classes_in_balanced)

# # Filter X and y
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

Classes in balanced dataset: {np.int64(0), np.int64(1), np.int64(2), np.int64(3)}


# Reduce class indexes by 1

In [11]:
#check if any label "0" is present in y
if (y == 0).any().any():
    print("Label '0' is present in y")
else:
    print("Label '0' is not present in y")

Label '0' is not present in y


In [13]:
# reduce y by 1
y['Class'] = y['Class'] - 1

# Remove gaps in labels

In [57]:
# remove gaps in labels, as in if there is a label "0" and then a label "2", turn the 2 into a 1
y = y.apply(lambda x: pd.factorize(x)[0])

# Dataset info

In [228]:
# number of samples
len(X)

28619

In [229]:
# number of classes (number of unique numbers in y)
len(np.unique(y.values))

2

In [230]:
# Convert y to int if needed
class_counts = y.value_counts()
print(class_counts)

is_sarcastic
0               14985
1               13634
Name: count, dtype: int64


In [231]:
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
print(class_percentages_sorted)

0    52.36032
1    47.63968
Name: proportion, dtype: float64


# Functions

In [232]:
def get_least_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    bottom_percent_count = int(np.ceil(num_classes * percent))
    least_represented_classes = class_percentages_sorted.tail(bottom_percent_count).index.tolist()
    return least_represented_classes

In [233]:
def get_most_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    top_percent_count = int(np.ceil(num_classes * percent))
    most_represented_classes = class_percentages_sorted.head(top_percent_count).index.tolist()
    return most_represented_classes

In [234]:
def show_summary(X, y):
    print("Number of samples:", len(X))
    print("Number of classes:", len(np.unique(y.values)))

    class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
    
    highest_class_percentage = class_percentages_sorted.iloc[0]
    lowest_class_percentage = class_percentages_sorted.iloc[-1]
    
    if lowest_class_percentage != 0:
        ratio = highest_class_percentage / lowest_class_percentage
        print(f"Times higher: {round(ratio, 2)}")
    else:
        ratio = np.nan
        print("The lowest class percentage is 0, cannot calculate ratio.")
        
    average_class_percentage = class_percentages_sorted.mean()
    class_percentages_std = class_percentages_sorted.std()

    cov = class_percentages_std / average_class_percentage
    print(f"CoV: {round(cov, 3)}")
    
    print(class_percentages_sorted)
    
    return class_percentages_sorted

In [235]:
def remove_class(X, y, chosen_class_index):
    # If DataFrame, select the first column
    if isinstance(y, pd.DataFrame):
        y_col = y.columns[0]
        y_series = y[y_col]
    else:
        y_series = y


    print(f"Removing class: {chosen_class_index}")
    
    # Create mask from 1D series
    mask = y_series != chosen_class_index

    # Filter X and y using 1D mask
    return X.loc[mask], y.loc[mask]


In [236]:
# removes SAMPLES
def remove_few_samples_of_class(X, y, class_index, step):
    indices_to_remove = y[y.values.flatten() == class_index].index[:step]
    X_trimmed = X.drop(indices_to_remove)
    y_trimmed = y.drop(indices_to_remove)
    
    return X_trimmed, y_trimmed

In [237]:
def remove_samples_from_all_classes(X, y, step):
    class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
    for chosen_class in class_percentages_sorted.index:
        X, y = remove_few_samples_of_class(X, y, chosen_class, step=step)   
    return X, y

# Dataset trimming

In [23]:
some_classes = get_most_represented_class_names(class_percentages_sorted, 0.8)
some_classes

[1, 0]

In [ ]:
# remove top class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[0]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

In [166]:
# remove bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-1]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

Removing class: 7
Number of samples: 175000
Number of classes: 7
Times higher: 1.0
CoV: 0.0
0    14.285714
1    14.285714
2    14.285714
3    14.285714
4    14.285714
5    14.285714
6    14.285714
Name: proportion, dtype: float64


In [434]:
# remove samples from top class

class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[0]
X,y = remove_few_samples_of_class(X, y, chosen_class, step=1)

class_percentages_sorted = show_summary(X, y)

Number of samples: 4784
Number of classes: 2
Times higher: 3.67
CoV: 0.809
0    78.595318
1    21.404682
Name: proportion, dtype: float64


In [424]:
# remove samples from bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-1]

X,y = remove_few_samples_of_class(X, y, chosen_class, step=100)
class_percentages_sorted = show_summary(X, y)

Number of samples: 5091
Number of classes: 2
Times higher: 3.97
CoV: 0.845
0    79.886073
1    20.113927
Name: proportion, dtype: float64


In [344]:
X,y = remove_samples_from_all_classes(X, y, step=200)
class_percentages_sorted = show_summary(X, y)

Number of samples: 14561
Number of classes: 2
Times higher: 4.16
CoV: 0.866
0    80.605728
1    19.394272
Name: proportion, dtype: float64


# Trimmed dataset - info

In [436]:
class_percentages_sorted = show_summary(X, y)

Number of samples: 4784
Number of classes: 2
Times higher: 3.67
CoV: 0.809
0    78.595318
1    21.404682
Name: proportion, dtype: float64


# Save

### Basic

In [1847]:
X.to_csv(DATASET_DIR / "X.csv", index=False)
y.to_csv(DATASET_DIR / "y.csv", index=False)

In [1848]:
class_percentages_sorted.to_csv(DATASET_DIR / "class_percentages.csv", header=False)

### Balanced

In [216]:
DATASET_DIR_BALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_BALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_BALANCED / "y.csv", index=False)

In [217]:
class_percentages_sorted.to_csv(DATASET_DIR_BALANCED / "class_percentages.csv", header=False)

### Imbalanced

In [439]:
DATASET_DIR_IMBALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_IMBALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_IMBALANCED / "y.csv", index=False)

In [440]:
class_percentages_sorted.to_csv(DATASET_DIR_IMBALANCED / "class_percentages.csv", header=False)